In [1]:
# Example:

# filename,width,height,class,xmin,ymin,xmax,ymax
## name, col(x), row(y)
# 34.jpg,236,241,ruby,134,48,203,118
# 34.jpg,236,241,weiss,79,18,153,106
# 35.jpg,236,189,ruby,79,23,132,76
# 35.jpg,236,189,weiss,111,11,163,72
# 37.jpg,236,295,weiss,31,35,123,129
# 37.jpg,236,295,ruby,112,22,203,116
# 36.jpg,622,350,ruby,128,22,309,226
# 36.jpg,622,350,weiss,321,1,508,148


In [36]:
import scipy
import os
import cv2
import csv
import random
import shutil
from scipy.io import loadmat

In [119]:
def oneOutTen():
    if random.randrange(0, 10) == 3:
        return True
    else:
        return False
    
def standardLabel(labelInput):
    #name tlx tly brx bry
    result = labelInput[0].split('    ')
    return result[0], result[1], result[2], result[3], result[4]

def cmp_img(a, b):
    a = os.path.splitext(a)[0].split('_')
    b = os.path.splitext(b)[0].split('_')
    
    a_body, a_num = a[1], a[2]
    b_body, b_num = b[1], b[2]
    
    if a_body > b_body:
        return 1
   
    if a_body < b_body:
        return -1
    
    if int(a_num) > int(b_num):
        return 1
    
    if int(a_num) < int(b_num):
        return -1
    
    return 0

In [ ]:
width = 640
height = 480

labelInputs = iter(sorted(['I_Avenue', 'I_BasketballField', 'I_Chinesebook', 'I_ClassRoom', 'I_ComputerScreen']))
data_type = 'test'

imgList = os.listdir(data_type)
imgList.sort(cmp=cmp_img)
print imgList[:50]
totalImg = len(imgList)
print 'total images: %s' % totalImg

currentLabel = labelInputs.next()
labelStream = open('label/%s_label.txt' % currentLabel, 'r')
reader = csv.reader(labelStream)

with open('deep_finger_%s.csv' % data_type, 'wb') as fout:
    writer = csv.writer(fout)
    writer.writerow(['filename','width','height','class','xmin','ymin','xmax','ymax'])

    counter = 0
    for item in imgList:
        counter += 1
        if counter % 5000 == 0:
            print counter
        if counter == totalImg:
            print counter
            
        while not item.startswith(currentLabel):
            currentLabel = labelInputs.next()
            labelStream.close()
            labelStream = open('label/%s_label.txt' % currentLabel, 'r')
            reader = csv.reader(labelStream)

        name, tlx, tly, brx, bry = standardLabel(reader.next())
        try:
            while not name == item:
                name, tlx, tly, brx, bry = standardLabel(reader.next())
        except StopIteration as e:
            print e
            print name
            print item
            raise
        
        writer.writerow([item, width, height, 'hand', int(float(tlx)*width), int(float(tly)*height), 
                         int(float(brx)*width), int(float(bry)*height)])

In [137]:
from PIL import Image

data_type = 'train'

print len(os.listdir(data_type))
counter = 0

for item in os.listdir(data_type):
    counter += 1
    if counter % 1000 == 0:
        print counter
    im = Image.open('%s/%s' % (data_type, item))
    rgb_im = im.convert('RGB')
    
    rgb_im.save('%s_jpg/%s.jpg' % (data_type, os.path.splitext(item)[0]))

16114
True


In [139]:
from PIL import Image

counter = {
    'I_Avenue': 0, 
    'I_BasketballField': 0, 
    'I_Chinesebook': 0, 
    'I_ClassRoom': 0, 
    'I_ComputerScreen': 0
}

def convert_img(imgCat):
    for item in os.listdir(data_type):
        if not item.startswith(imgCat):
            continue
        
        counter[imgCat] += 1
        if counter[imgCat] % 1000 == 0:
            print '%s: %s' % (imgCat, counter[imgCat])
            
        im = Image.open('%s/%s' % (data_type, item))
        rgb_im = im.convert('RGB')
        rgb_im.save('%s_jpg/%s.jpg' % (data_type, os.path.splitext(item)[0]))

In [142]:
from multiprocessing import Pool
pool = Pool(processes=5)
results = pool.map(convert_img, counter.keys())


I_ComputerScreen: 1000
I_Chinesebook: 1000
I_ClassRoom: 1000
I_Avenue: 1000
I_BasketballField: 1000
I_ComputerScreen: 2000
I_Chinesebook: 2000
I_ClassRoom: 2000
I_Avenue: 2000
I_BasketballField: 2000
I_ComputerScreen: 3000
I_ClassRoom: 3000
I_Avenue: 3000
I_ComputerScreen: 4000
